In [97]:
import numpy as np

import networkx as nx

from IPython.display import HTML

import matplotlib.cm as cm
cmap = cm.get_cmap('gist_rainbow', 100)

from matplotlib import animation, rc

import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['animation.html'] = 'jshtml'
plt.rcParams['animation.embed_limit'] = 2**128

import warnings
warnings.filterwarnings("ignore")

# from celluloid import Camera

In [98]:
def calc_Q(G):
    m = sum(sum(G))
    inner_sum = 0
    for i in range(len(G)):
        A_i_i  = G[i,i]
        k_i_out = sum(G[i])
        k_i_in  = sum(G.T[i])
        inner_sum += (A_i_i - ((k_i_out*k_i_in)/m))

    return inner_sum / m

In [102]:
def naive_greedy_itteration(G, Q, comms, itter, ecolors, ncolors):
    best_delta_Q = -1
    for i in range(len(G)):
        for j in range(len(G)):
            if i >= j:
                continue
            itter += 1
            ecolors[itter] = ecolors[itter - 1].copy()
            ecolors[itter][(i,j)] = 'white'
            ncolors[itter] = ncolors[itter - 1].copy()
            # ncolors[itter]
            comms[itter] = comms[itter - 1].copy()

            prefix = G[0:i,:]
            join = (G[i,:] + G[j,:]).reshape(1,len(G))
            interim = G[i+1:j,:]
            postfix = G[j+1:,:]
            if prefix.shape == (0,len(G)):
                cj = np.vstack((join, interim, postfix))
            elif interim.shape == (0,len(G)):
                cj = np.vstack((prefix, join, postfix))
            elif postfix.shape == (0,len(G)):
                cj = np.vstack((prefix, join, interim))
                
            prefix = cj[:,0:i]
            join = (cj[:,i] + cj[:,j]).reshape(len(cj),1)
            interim = cj[:,i+1:j]
            postfix = cj[:,j+1:]
            if prefix.shape == (len(cj),0):
                crj = np.hstack((join, interim, postfix))
            if interim.shape == (len(cj),0):
                crj = np.hstack((prefix, join, postfix))
            if postfix.shape == (len(cj),0):
                crj = np.hstack((prefix, join, interim))
            
            delta_Q = calc_Q(crj) - Q
            if delta_Q > best_delta_Q:
                best_delta_Q = delta_Q
                G_prime = crj
                best_i = i
                best_j = j

            itter += 1
            ecolors[itter] = ecolors[itter - 1].copy()
            ecolors[itter][(i,j)] = 'purple'
            ncolors[itter] = ncolors[itter - 1].copy()
            # ncolors[itter]
            comms[itter] = comms[itter - 1].copy()

    comms[itter] = comms[itter - 1].copy()
    comms[itter][best_i] = comms[itter][best_i].union(comms[itter].pop(best_j))
    return G_prime, best_delta_Q, comms, itter, ecolors, ncolors

In [114]:
N = 25
np.random.seed(12)
b = np.random.randint(low=0,high=10,size=(N,N)) 
G_ = nx.from_numpy_array(b + b.T)
# p = nx.spring_layout(G_)
p = nx.spectral_layout(G_)


G = (b + b.T)
Q = calc_Q(G)
itter = 0
comms = {}
comms[itter] = [{n} for n in range(len(G))]
best_fit = (Q, itter)

ecolors = {}
ecolors[itter] = {edge:'purple' for edge in G_.edges()}

ncolors = {}
ncolors[itter] = {node:'white' for node in G_.nodes()}

c_diff = 1 / N
c = 0
for community in comms[0]:
    for node in community:
        ncolors[itter][node] = cmap(c)
    c += c_diff

bitter = 0
while bitter < (N - 1):
    bitter += 1
    G, dQ, comms, itter, ecolors, ncolors = naive_greedy_itteration(G, Q, comms, itter, ecolors, ncolors)
    Q += dQ
    if best_fit[0] <= Q:
        best_fit = (Q, itter)

    c_diff = 1 / N
    c = 0
    for community in comms[best_fit[1]]:
        for node in community:
            ncolors[itter][node] = cmap(c)
        c += c_diff


fig = plt.figure(figsize=(16,9))

def animate(iter):
    fig.clear()
    nx.draw_networkx_edges(G_, pos=p, edgelist=G_.edges(), edge_color=ecolors[iter].values(), alpha=0.5)
    nx.draw_networkx_nodes(G_, pos=p, node_size=50, node_color=list(ncolors[iter].values()), alpha=0.75)
    # print(iter)
    return fig

anim = animation.FuncAnimation(fig, animate, frames=itter, interval=50, blit=False, repeat=False, cache_frame_data=False)
HTML(anim.to_jshtml())